# GLM Example
Generalized linear models are a generalization of linear regression. In linear regression, responses are modeled as coming from gaussian distributions, each of equal variance, with each distribution's mean given by a linear combination of the predictor variables. GLMs allow other distributions where the variance might vary with the mean, and they allow the mean to be given by a function of the linear predictor instead of taking the linear predictor directly.

## Boston housing prices
We'll demonstrate by using GLMs to predict the median price of a home in Boston.  The variables are crime rate, zoning information,
proportion of non-retail business, etc.  This dataset has median prices in Boston for 1972.  Even though the data is pretty old, the methodology for analytics is valid for more recent datasets.

The dataset is from Kaggle. https://www.kaggle.com/c/boston-housing. For tutorials use only.

## Housing Values in Suburbs of Boston in 1972

The <font color='red'>medv</font> variable is the target variable.
### Data description
The Boston data frame has 506 rows and 14 columns.
This data frame contains the following columns:
1. __crim__: per capita crime rate by town.
2. __zn__: proportion of residential land zoned for lots over 25,000 sq.ft.
3. __indus__: proportion of non-retail business acres per town.
4. __chas__: Charles River dummy variable (1 if tract bounds river; 0 otherwise).
5. __nox__: nitrogen oxides concentration (parts per 10 million).
6. __rm__: average number of rooms per dwelling.
7. __age__: proportion of owner-occupied units built prior to 1940.
8. __dis__: weighted mean of distances to five Boston employment centres.
9. __rad__: index of accessibility to radial highways.
10. __tax__: full-value property-tax rate per \$10000
11. __ptratio__: pupil-teacher ratio by town.
12. __black__: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.
13. __lstat__: lower status of the population (percent).
14. __medv__: median value of owner-occupied homes in $1000s.
</td></tr></table>

### Factoids
The prices in Boston across years are below.  If we had a historical dataset, an analysis could be done to account for the macro trends as well.

The second graph shows the intuition we have with respect to prices in relation to crime rate.  It is expected that house prices will be lower in areas where crime rates are higher.

The third figure is a chart showing how inflation may affect prices.  So, for deeper analysis and prediction, we may want to consider inflation.

In this notebook, these factors are not considered.  They are here to demonstrate the need for deep domain analysis.

<table><tr>
<td><img src="images/boston_prices_by_year.png" alt="Boston home prices" title="Boston housing prices" style="float:left;" /></td>
<td><img src="images/Crime-Rate-and-Median-House-Prices.png" alt="Boston home prices" title="Boston housing prices"  /></td>
<td><img src="images/Inflation_Adjusted_Housing_Prices_1890_2006.jpg" alt="Inflation adjusted prices" title="Inflation adjusted prices" style="float:left;" />
</td></tr></table>


In this notebook, we will use the dataset for Boston housing prices and predict the price based on numerous factors.

In [1]:
from hana_ml import dataframe
from hana_ml.algorithms.pal import regression
import numpy as np
import matplotlib.pyplot as plt
import logging

## Load data
The data is loaded into 4 tables, for full, training, validation, and test sets:
<li>BOSTON_HOUSING_PRICES</li>
<li>BOSTON_HOUSING_PRICES_TRAINING</li>
<li>BOSTON_HOUSING_PRICES_VALIDATION</li>
<li>BOSTON_HOUSING_PRICES_TEST</li>

To do that, a connection is created and passed to the loader.

There is a config file, config/e2edata.ini that controls the connection parameters and whether or not to reload the data from scratch.  In case the data is already loaded, there would be no need to load the data.  A sample section is below.  If the config parameter, reload_data is true then the tables for test, training, and validation are (re-)created and data inserted into them.

#########################<br>
[hana]<br>
url=host.sjc.sap.corp<br>
user=username<br>
passwd=userpassword<br>
port=3xx15<br>
<br>
[bostonhousingdataset]<br>
reload_data=true
#########################<br>

In [2]:
from data_load_utils import DataSets, Settings
url, port, user, pwd = Settings.load_config("../../config/e2edata.ini")
connection_context = dataframe.ConnectionContext(url, port, user, pwd)
full_tbl, training_tbl, validation_tbl, test_tbl = DataSets.load_boston_housing_data(connection_context)

Table BOSTON_HOUSING_PRICES exists and data exists


# Create Data Frames
Create the data frames for the full, test, training, and validation sets.

We'll also do some data exploration.

## Define Datasets - Training, validation, and test sets
Data frames represent data in HANA and HANA-side queries on that data, so computation on large data sets in HANA can happen in HANA.  Trying to bring the entire data set into the client may be impractical or impossible for large data sets.

The original/full dataset is split into training, test and validation sets.  In the example below, they reside in different tables.

In [3]:
full_set = connection_context.table(full_tbl)
training_set = connection_context.table(training_tbl)
validation_set = connection_context.table(validation_tbl)
test_set = connection_context.table(test_tbl)

## Simple Exploration
Let us look at the number of rows in the data set.

In [4]:
print('Number of rows in full set: {}'.format(full_set.count()))
print('Number of rows in training set: {}'.format(training_set.count()))
print('Number of rows in validation set: {}'.format(validation_set.count()))
print('Number of rows in test set: {}'.format(test_set.count()))

Number of rows in full set: 506
Number of rows in training set: 253
Number of rows in validation set: 202
Number of rows in test set: 51


### Let's look at the columns

In [5]:
print(full_set.columns)

['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'BLACK', 'LSTAT', 'MEDV', 'ID']


### Let's look at the data types

In [6]:
full_set.dtypes()

[('CRIM', 'DECIMAL', 12, 12, 12, 5),
 ('ZN', 'DECIMAL', 7, 7, 7, 3),
 ('INDUS', 'DECIMAL', 7, 7, 7, 2),
 ('CHAS', 'SMALLINT', 5, 5, 5, 0),
 ('NOX', 'DECIMAL', 10, 10, 10, 4),
 ('RM', 'DECIMAL', 8, 8, 8, 3),
 ('AGE', 'DECIMAL', 7, 7, 7, 3),
 ('DIS', 'DECIMAL', 11, 11, 11, 4),
 ('RAD', 'TINYINT', 3, 3, 3, 0),
 ('TAX', 'SMALLINT', 5, 5, 5, 0),
 ('PTRATIO', 'DECIMAL', 6, 6, 6, 2),
 ('BLACK', 'DECIMAL', 9, 9, 9, 3),
 ('LSTAT', 'DECIMAL', 7, 7, 7, 2),
 ('MEDV', 'DECIMAL', 6, 6, 6, 2),
 ('ID', 'INT', 10, 10, 10, 0)]

### Set up the features and labels for the model

In [7]:
features=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'BLACK', 'LSTAT']
label='MEDV'

# Create model using training data
The first GLM we create will use the default settings of `family='gaussian'` and `link='identity'`. A GLM using these settings is equivalent to linear regression.

## Preprocessing
SAP HANA Predictive Analytics Library takes DOUBLE and INTEGER data types for most numeric types.  Since we have DECIMALs and TINYINTs in our data set, we cast them to the types required by PAL.

In [8]:
# Cast to correct types so PAL can consume it.
dfts = training_set.cast(['CRIM', "ZN", "INDUS", "NOX", "RM", "AGE", "DIS", "PTRATIO", "BLACK", "LSTAT", "MEDV"], "DOUBLE")
dfts = dfts.cast(["CHAS", "RAD", "TAX"], "INTEGER")
dfts = dfts.to_head("ID")
dfts.head(5).collect()

,ID,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,BLACK,LSTAT,MEDV
0,196,0.01381,80.0,0.46,0,0.422,7.875,32.0,5.6484,4,255,14.4,394.23,2.97,50.0
1,124,0.15038,0.0,25.65,0,0.581,5.856,97.0,1.9444,2,188,19.1,370.31,25.41,17.3
2,60,0.10328,25.0,5.13,0,0.453,5.927,47.2,6.9320,8,284,19.7,396.90,9.22,19.6
3,76,0.09512,0.0,12.83,0,0.437,6.286,45.0,4.5026,5,398,18.7,383.23,8.94,21.4
4,482,5.70818,0.0,18.10,0,0.532,6.750,74.9,3.3317,24,666,20.2,393.07,7.74,23.7


## Create the model

In [9]:
linear_model = regression.GLM()
linear_model.fit(dfts, key='ID', features=features, label=label)

Let's see how well this model does. We'll compute the R^2 score of its predictions on the test set.

In [10]:
df_test = test_set.cast(['CRIM', "ZN", "INDUS", "NOX", "RM", "AGE", "DIS", "PTRATIO", "BLACK", "LSTAT", "MEDV"], "DOUBLE")
df_test = df_test.cast(["CHAS", "RAD", "TAX"], "INTEGER")
df_test = df_test.to_head("ID")
linear_model.score(df_test, key='ID', features=features, label=label)

0.7192430873425482

Let's try a few others. We'll experiment with gamma distributions and inverse or logarithmic link functions.

In [11]:
gaussian_idlink = regression.GLM(family='gaussian', link='identity')
gamma_invlink = regression.GLM(family='gamma', link='inverse')
gaussian_invlink = regression.GLM(family='gaussian', link='inverse')
gamma_loglink = regression.GLM(family='gamma', link='log')
gaussian_loglink = regression.GLM(family='gaussian', link='log')
for model in [gaussian_idlink, gamma_invlink, gaussian_invlink, gamma_loglink, gaussian_loglink]:
    model.fit(dfts, key='ID', features=features, label=label)
    print(model.score(df_test, key='ID', features=features, label=label))

0.7192430873425482
0.8387921104804832
0.8433749423700786
0.8083099068011791
0.8014951981496059


An inverse link seems to help model our data better. Our error terms seem to be modeled better by our original choice of `family='gaussian'` than `family='gamma'`.